### Rechner-App: Beispiel zu Lambda-Expressions und Callbacks
Wir wollen eine einfache Taschenrechner-App programmieren.


**L&ouml;sungsansatz**: Wir kreieren Buttons f&uuml;r alle Ziffern und die g&auml;ngigen Operation. Der Button tr&auml;gt die Ziffer oder das Operationssymbol als Label.
Weiter wird jedem Button eine Funktion `f(bt)` (Callback) zugewiesen, welche beim Dr&uuml;cken des Buttons aufgerufen wird. Als Argument wird der Funktion das Buttonobjekt &uuml;bergeben, welches wir ignorieren.


Wird ein Button gedr&uuml;ckt, so h&auml;ngt die aufgerufene Funktion 
das Symbol auf dem Button an eine Liste an und zeigt diese Liste im Display an.

Wird `=` gedr&uuml;ckt, so nutzen wir Pythons `eval` Funktion, um den Ausdruck im Display auszuwerten.

***
**Widgets und Zugeh&ouml;riges importieren**  
siehe z.B. 
- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-Grid-layout 
- https://ipywidgets.readthedocs.io/en/latest/examples/Layout%20Templates.html  
- https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
***

In [5]:
from ipywidgets import Button, GridBox, Text, Output, HBox, HTML,\
                       Layout, ButtonStyle

***
**Callbackfactory, Ausdruck im Display auswerten**
***

In [2]:
# update display-widget
def update_display(out, display_chars):
    ''' clear display-widget out and display display_chars
    
        out: display widget
        display_chars: list of characters to be dislayed
    '''
    out.clear_output()
    text = ''.join(display_chars)
    style = 'color: blue; font-size: 30px; line-height: 1.6; float: right;'
    html = '<span style="{style}">{text}</span>'\
           .format(style=style, text=text)
    
    # display html in the widget out
    with out:
        display(HTML(html))
        
def callback(action, out, display_chars):
    ''' returns a callback
        
    action:        function 
    out:           output-widget
    display_chars: list of characters, 
    '''    
    def f(bt):
        action()
        update_display(out, display_chars)
    return f

# evaluate displayed expression 
def eval_expression(display_chars):
    '''evalutate the expression formed by the 
       characters in display_chars
    '''   
    exp = ''.join(display_chars)
    try: 
        res = eval(exp)
    except (SyntaxError, ZeroDivisionError, OverflowError) as e:
        res = type(e).__name__
        
    res = '{:16.8g}'.format(res)
    display_chars[:] = str(res)

***
**Widgets stylen und anordnen**
***

In [3]:
layout_out = {'border': '1px solid black',  'width': '100%', 'height':'45px',}    
out  = Output(layout=layout_out)
disp = HBox([out], layout=Layout(width='auto',  height='auto', grid_area='header'))

buttons = [Button(layout=Layout(width='auto', height='auto'),
                         style=ButtonStyle(button_color='darkseagreen')) for i in range(20)
                 ]

app = GridBox(children=[disp] + buttons,
             layout=Layout(width='auto',
                           grid_template_columns =' 50px 50px 50px 50px 50px',
                           grid_template_rows    = '50px 50px 50px 50px 50px',
                           grid_gap='5px 10px',
                           grid_template_areas=''' "header header header header header" '''
                          )
             )
# display(app) # uncomment to see unlabelled buttons
               # then recomment and reexcecute   

***
**F&uuml;gen wir alles zusammen**:
***

In [8]:
# characters in calculator's display
display_chars = []

# dictionary with character-action pairs
# note the use of default-arguments!
char_action   = {ch: lambda x=ch: display_chars.append(x)\
                     for ch in '789()456+-123*/0.⎚^='}

# overwrite some of these action
char_action['^'] = lambda: display_chars.append('**')
char_action['⎚'] = display_chars.clear
char_action['='] = lambda: eval_expression(display_chars)

# label the buttons, attach callbacks to the buttons
for bt, (ch, f) in zip(buttons, char_action.items()):
    # bt._click_handlers.callbacks = []
    bt.description = ch # label the button bt
    bt.on_click(callback(f, out, display_chars)) # attach callback to bt

display(app)

GridBox(children=(HBox(children=(Output(layout=Layout(border='1px solid black', height='45px', width='100%'), …